# Usage of pretrained model (VGG16)

## Import modules

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

import tensorflow as tf
tf.enable_eager_execution()

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
print('TensorFlow version: {}'.format(tf.__version__))

In [ ]:
# if you have a memeory trouble then uncomment it
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

## Usage `VGG16`
* [code link](https://github.com/keras-team/keras-applications/blob/master/keras_applications/vgg16.py)
* [document link](https://keras.io/applications/#vgg16)

```python
def VGG16(include_top=True,
          weights='imagenet',
          input_tensor=None,
          input_shape=None,
          pooling=None,
          classes=1000,
          **kwargs):
    """Instantiates the VGG16 architecture.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        include_top: whether to include the 3 fully-connected
            layers at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(224, 224, 3)`
            (with `channels_last` data format)
            or `(3, 224, 224)` (with `channels_first` data format).
            It should have exactly 3 input channels,
            and width and height should be no smaller than 32.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
```

### General method (`include_top=True`)

In [ ]:
vgg16_1 = tf.keras.applications.VGG16()

In [ ]:
vgg16_1.summary()

### General method (`include_top=False`)

In [ ]:
vgg16_2 = tf.keras.applications.VGG16(include_top=False,
                                      input_shape=(150, 150, 3))

In [ ]:
vgg16_2.summary()

#### Read a image

In [ ]:
def vgg_preprocessing(image):
  """image preprocessing

  Args:
    image (PIL image): image with shape [height, width, channels]
    
  Returns:
    image (np.float32): vgg preprocessed image with 4-rank tensor shape [1, height, width, channels] applied by mean_image_subtraction
  """
  norm_means = np.array([123.68, 116.779, 103.939])
  vgg_image_size = 224
  image = image.resize((vgg_image_size, vgg_image_size))
  image = np.asarray(image)
  image = image.astype(np.float32)
  image -= norm_means
  
  return image

In [ ]:
my_image = Image.open('../../input_data/cat2.jpg')
plt.imshow(my_image)

In [ ]:
my_image = vgg_preprocessing(my_image)
my_image = np.expand_dims(my_image, axis=0)

### Inference `my_image` through VGG16

In [ ]:
prediction = vgg16_1(my_image)

In [ ]:
prediction.shape

In [ ]:
pred_index = tf.argmax(prediction, axis=1)[0]
print('prediction index: {}'.format(pred_index))

### Read a label file

In [ ]:
category_names = []
with open('../../input_data/imagenet.classname.txt', 'r') as f:
  for line in f.readlines():
    category_names.append(line.strip())

In [ ]:
category_names[pred_index]

### Observe all activation maps

In [ ]:
layer_outputs = [layer.output for layer in vgg16_1.layers[1:]]
for layer in layer_outputs:
  print(layer)

### Observe all training variables

In [ ]:
for var in vgg16_1.variables:
  print(var.name)

In [ ]:
vgg16_1.variables[0].shape

In [ ]:
vgg16_1.layers[1].kernel.shape

In [ ]:
vgg16_1.variables[0].numpy() == vgg16_1.layers[1].kernel.numpy()

### Extract all activation maps (feature maps)

In [ ]:
from tensorflow.keras import models

In [ ]:
activation_model = models.Model(inputs=vgg16_1.input, outputs=layer_outputs)

In [ ]:
activations = activation_model.predict(my_image)

In [ ]:
conv1_1 = activations[0]
conv2_1 = activations[2]
conv3_2 = activations[6]
conv4_3 = activations[11]
conv5_3 = activations[15]

### Print feature maps

In [ ]:
def print_all_feature_maps(layer, layer_name):
  """Print all feature maps
    This code is borrowed from "Deep Learning with Python" (by F. Chollet)
  
  Args:
    layer (4-rank Tensor): feature maps
    layer_name (string): name of feature maps
    
  Returns:
    print all feature maps
  """
  num_features = layer.shape[-1]
  size = layer.shape[1]
  images_per_row = 16
  for feature_map in range(num_features):
    num_cols = num_features // images_per_row
    display_grid = np.zeros((size * num_cols, images_per_row * size))

    for col in range(num_cols):
      for row in range(images_per_row):
        channel_image = layer[0,:,:,col * images_per_row + row]

        channel_image -= channel_image.mean()
        channel_image /= channel_image.std()
        channel_image *= 64
        channel_image += 128
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')

        display_grid[col * size : (col + 1) * size,
                     row * size : (row + 1) * size] = channel_image

  scale = 1. / size
  plt.figure(figsize=(scale * display_grid.shape[1],
                      scale * display_grid.shape[0]))
  plt.title(layer_name)
  plt.grid(False)
  plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [ ]:
print_all_feature_maps(conv1_1, 'conv1_1')

In [ ]:
print_all_feature_maps(conv2_1, 'conv2_1')

In [ ]:
print_all_feature_maps(conv3_2, 'conv3_2')

In [ ]:
print_all_feature_maps(conv4_3, 'conv4_3')

In [ ]:
print_all_feature_maps(conv5_3, 'conv5_3')